![image](https://github.com/eWaterCycle/ewatercycle/raw/main/docs/examples/logo.png)

# Generate CMIP6 forcing in eWaterCycle with ESMValTool, for SSP126, SSP245 and SSP585 scenarios

This notebooks shows how to generate CMIP6 forcing data for Wflow model [ESMValTool hydrological recipes](https://docs.esmvaltool.org/en/latest/recipes/recipe_hydrology.html#wflow). More information about data, configuration and installation instructions can be found in the [System setup](https://ewatercycle.readthedocs.io/en/latest/system_setup.html) in the eWaterCycle documentation.

In [1]:
import logging
import warnings
from pathlib import Path

warnings.filterwarnings("ignore", category=UserWarning)

logger = logging.getLogger("esmvalcore")
logger.setLevel(logging.WARNING)

In [2]:
import xarray as xr
from rich import print
from datetime import datetime, timedelta
import pandas as pd
import re
import json
import logging
import warnings
from ewatercycle import CFG
import ewatercycle.forcing
import ewatercycle.models
import ewatercycle.parameter_sets
import ewatercycle.analysis
from ewatercycle.testing.fixtures import rhine_shape


warnings.filterwarnings("ignore", category=UserWarning)
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger("esmvalcore")


OSError: [Errno 5] Input/output error: '/mnt/data/observation/grdc/dailies'

CMIP forcing for Wflow is created using the ESMValTool recipe. It produces three files of the SSP-Pathways that contain three variables: temperature, precipitation, and potential evapotranspiration. You can set the start and end date, and the region. See [eWaterCycle documentation](https://ewatercycle.readthedocs.io/en/latest/apidocs/ewatercycle.forcing.html#submodules) for more information. 

To download `wflow_dem.map`, see the [instructions](https://ewatercycle.readthedocs.io/en/latest/system_setup.html#download-example-parameter-sets).
To find working CMIP data, using the step_0_select_CMIP_forcing notebook can be used, this notebook creates a json file that stores all working sets for Wflow CMIP forcing. 
Generate forcing data requires to specify the same start_time and end_time used in the Wflow notebook in the cell below, then specify the working CMIP sets found using the selecting notebook.

In [6]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta


start_time="2050-01-01" # Use the same start_time as in the Wflow notebook
end_time="2075-12-31"   # Use the same end_time as in the Wflow notebook
start_time+"T00:00:00Z", end_time+"T00:00:00Z"

ssp126 = {
    "project": "CMIP6",
    "activity": "ScenarioMIP",
    "exp": "ssp126",
    "dataset": "MPI-ESM1-2-LR",
    "ensemble": "r10i1p1f1",
    "grid": "*",
}

ssp245 = {
    "project": "CMIP6",
    "activity": "ScenarioMIP",
    "exp": "ssp245",
    "dataset": "MPI-ESM1-2-LR",
    "ensemble": "r10i1p1f1",
    "grid": "*",
}
ssp585 = {
    "project": "CMIP6",
    "activity": "ScenarioMIP",
    "exp": "ssp585",
    "dataset": "MPI-ESM1-2-LR",
    "ensemble": "r10i1p1f1",
    "grid": "*",
}


# Starting time minus 2 months to create time for model spinup
start_time = datetime.strptime(start_time, "%Y-%m-%d")
end_time = datetime.strptime(end_time, "%Y-%m-%d")

start_time = start_time - relativedelta(months=2)
print(start_time, end_time)

2049-11-01 00:00:00 2075-12-31 00:00:00

In [15]:
configs = [ssp126, ssp245, ssp585]

# List to store forcings and directories
wflow_forcings = []
directories = []

# Generate forcings in a loop
for i, config in enumerate(configs, 1):
    forcing = ewatercycle.forcing.sources["WflowForcing"].generate(
        dataset=config,
        start_time=start_time+"T00:00:00Z",
        end_time=end_time+"T00:00:00Z",
        shape="./Rhine/Rhine.shp",
        dem_file=str(Path().absolute() / "./wflow_dem.map"),
    )
    wflow_forcings.append(forcing)
    print(f'wflow_forcing_c{i} = {forcing}')    
    
    # Extract the directory from the forcing object
    directory_output = f"directory=PosixPath('{forcing.directory}')"
    match = re.search(r"directory=PosixPath\('(.+?)'\)", directory_output)
    
    if match:
        directories.append(match.group(1))

# Printing the directory paths
for directory in directories:
    print(directory)

NameError: name 'ewatercycle' is not defined

The file will be named according to the set timespan in the beginning of this notebook

In [16]:
# Specify the file path
json_file_path = f"forcing_data_CMIP6_wflow_{start_time}_{end_time}.json"

# Write data to JSON file
data = {
    "start_time": start_time,
    "end_time": end_time,
    "directories": directories
}

with open(json_file_path, 'w') as file:
    json.dump(data, file, indent=4)

print(f"Data has been written to: {json_file_path}")

Data has been written to: forcing_data_CMIP6_wflow_2024-11-01 00:00:00_2050-12-31 00:00:00.json